In [1]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import pandas as pd
import numpy as np
import os, shutil, glob, os.path
from PIL import Image as pil_image
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# load VGG16 model

image.LOAD_TRUNCATED_IMAGES = True 
model = VGG16(weights='imagenet', include_top=False)

W0524 20:43:37.332023 140130645100352 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0524 20:43:37.345724 140130645100352 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0524 20:43:37.403120 140130645100352 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4420: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0524 20:43:37.432808 140130645100352 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4255: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0524 20:43:37.751728 140130645100352 deprecation_wrapp

In [3]:
badImgDir = './clusters/teste/0'
goodImgDir = './clusters/teste/1'


# get images
goodFiles = glob.glob(os.path.join(goodImgDir, '*.jpg'))
goodFiles.sort()

badFiles = glob.glob(os.path.join(badImgDir, '*.jpg'))
badFiles.sort()

In [4]:
# Using Keras to get data from image

def getFeature(file):

    features = []

    for i, imagepath in enumerate(file):
        print("Status:", i+1, "/", len(file), end="\r")

        img = image.load_img(imagepath, target_size=(256, 256))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        feat = np.array(model.predict(img_data))
        features.append(feat.flatten())

    return features

In [5]:
dfGood = pd.DataFrame(getFeature(goodFiles))

In [6]:
dfBad = pd.DataFrame(getFeature(badFiles))

In [7]:
goodTarget = []

for i in range(len(dfGood)):
    goodTarget.append(1)
    
badTarget = []

for i in range(len(dfBad)):
    badTarget.append(0)

In [8]:
dfGood["Target"] = goodTarget
dfBad["Target"] = badTarget

In [9]:
df = dfGood.append(dfBad)

In [10]:
#df.to_csv("imageDataSet.csv", encoding='utf-8', index=False)
corr = df.corr()

In [20]:
import math

last_corr = [1621, 5193, 5616, 6659, 8683, 9170, 9195, 9200, 9289, 9498, 9636, 9692, 10148, 10204, 10660, 11172, 11760, 12754, 12779, 13266, 13276, 13296, 13732, 13788, 14244, 14756, 15268, 15292, 16875, 17591, 17690, 17828, 17852, 18103, 18615, 18714, 18852, 19226, 19364, 21924, 21948, 22199, 22436, 22460, 22948, 23460, 25230, 25742, 26254, 26556, 26578, 27068, 32768]
good_corr = []
both_corr = []
neither_corr = []

for index, num in enumerate(corr["Target"]):
    if num > 0.5 or num < -0.5:
        good_corr.append(index)

for index in good_corr:
    if index in last_corr:
        both_corr.append(index)
    else:
        neither_corr.append(index)
        
        
print(both_corr)
print("----------------")
print(neither_corr)
print("----------------")
print(good_corr)


[1621, 5193, 8683, 9170, 9200, 9289, 9636, 9692, 10148, 10204, 10660, 11760, 12754, 12779, 13266, 13732, 13788, 14244, 14756, 15268, 16875, 17591, 17690, 17828, 17852, 18103, 18615, 18714, 18852, 19226, 19364, 21924, 21948, 22199, 22436, 22460, 22948, 23460, 25230, 25742, 26254, 26556, 27068, 32768]
----------------
[1109, 1111, 1623, 2133, 4693, 4695, 5207, 5319, 8777, 9392, 10825, 13385, 14921, 16969, 17316, 17481, 17723, 17870, 17884, 18202, 18340, 18747, 21687, 22711, 22810, 22972, 23004, 23484, 23739, 25275]
----------------
[1109, 1111, 1621, 1623, 2133, 4693, 4695, 5193, 5207, 5319, 8683, 8777, 9170, 9200, 9289, 9392, 9636, 9692, 10148, 10204, 10660, 10825, 11760, 12754, 12779, 13266, 13385, 13732, 13788, 14244, 14756, 14921, 15268, 16875, 16969, 17316, 17481, 17591, 17690, 17723, 17828, 17852, 17870, 17884, 18103, 18202, 18340, 18615, 18714, 18747, 18852, 19226, 19364, 21687, 21924, 21948, 22199, 22436, 22460, 22711, 22810, 22948, 22972, 23004, 23460, 23484, 23739, 25230, 25275